In [38]:
!pip uninstall numpy catboost -y
!pip install numpy catboost
!pip install lightgbm
!pip3 install catboost
!pip install xgboost
!pip install optuna

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Found existing installation: catboost 1.2.7
Uninstalling catboost-1.2.7:
  Successfully uninstalled catboost-1.2.7
Defaulting to user installation because normal site-packages is not writeable
  Using cached numpy-2.2.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.4 MB)
  Using cached catboost-1.2.7-cp310-cp310-manylinux2014_x86_64.whl (98.7 MB)
  Using cached numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.2 MB)
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 KB 6.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━

In [50]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
# new import statements
import lightgbm as lgb
import xgboost as xgb
import optuna

### Retailrocket recommender system dataset

Source: https://www.kaggle.com/retailrocket/ecommerce-dataset

Load the dataset files.

In [10]:
events = pd.read_csv(os.path.join('data', 'events.csv'))
item_properties = pd.read_csv(os.path.join('data', 'item_properties_part1.csv'))
category_tree = pd.read_csv(os.path.join('data', 'category_tree.csv'))

In [11]:
events

,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN
2,1433221999827,111016,view,318965,NaN
3,1433221955914,483717,view,253185,NaN
4,1433221337106,951259,view,367447,NaN
...,...,...,...,...,...
2756096,1438398785939,591435,view,261427,NaN
2756097,1438399813142,762376,view,115946,NaN
2756098,1438397820527,1251746,view,78144,NaN
2756099,1438398530703,1184451,view,283392,NaN


In [12]:
item_properties

,timestamp,itemid,property,value
0,1435460400000,460429,categoryid,1338
1,1441508400000,206783,888,1116713 960601 n277.200
2,1439089200000,395014,400,n552.000 639502 n720.000 424566
3,1431226800000,59481,790,n15360.000
4,1431831600000,156781,917,828513
...,...,...,...,...
10999994,1439694000000,86599,categoryid,618
10999995,1435460400000,153032,1066,n1020.000 424566
10999996,1440298800000,421788,888,35975 856003 37346
10999997,1437879600000,159792,400,n552.000 639502 n720.000 424566


In [13]:
category_tree

,categoryid,parentid
0,1016,213.0
1,809,169.0
2,570,9.0
3,1691,885.0
4,536,1691.0
...,...,...
1664,49,1125.0
1665,1112,630.0
1666,1336,745.0
1667,689,207.0


In [14]:
events['event_datetime'] = pd.to_datetime(events['timestamp'], unit='ms')
events['event_date'] = events['event_datetime'].dt.date
events

,timestamp,visitorid,event,itemid,transactionid,event_datetime,event_date
0,1433221332117,257597,view,355908,NaN,2015-06-02 05:02:12.117,2015-06-02
1,1433224214164,992329,view,248676,NaN,2015-06-02 05:50:14.164,2015-06-02
2,1433221999827,111016,view,318965,NaN,2015-06-02 05:13:19.827,2015-06-02
3,1433221955914,483717,view,253185,NaN,2015-06-02 05:12:35.914,2015-06-02
4,1433221337106,951259,view,367447,NaN,2015-06-02 05:02:17.106,2015-06-02
...,...,...,...,...,...,...,...
2756096,1438398785939,591435,view,261427,NaN,2015-08-01 03:13:05.939,2015-08-01
2756097,1438399813142,762376,view,115946,NaN,2015-08-01 03:30:13.142,2015-08-01
2756098,1438397820527,1251746,view,78144,NaN,2015-08-01 02:57:00.527,2015-08-01
2756099,1438398530703,1184451,view,283392,NaN,2015-08-01 03:08:50.703,2015-08-01


In [24]:
category_items = item_properties[item_properties['property'] == 'categoryid'][['itemid', 'value']]
# 'value' column contains the category IDs, so let's rename value to categoryid
category_items.columns = ['itemid', 'categoryid']  
category_items['categoryid'] = category_items['categoryid'].astype(int)
category_items

,itemid,categoryid
0,460429,1338
140,281245,1277
151,35575,1059
189,8313,1147
197,55102,47
...,...,...
10999880,441523,1167
10999917,250848,769
10999932,116380,1509
10999960,84186,209


In [25]:
events_with_categories = pd.merge(events, category_items, on="itemid", how="left")
events_with_categories

,timestamp,visitorid,event,itemid,transactionid,event_datetime,event_date,categoryid
0,1433221332117,257597,view,355908,NaN,2015-06-02 05:02:12.117,2015-06-02,1173.0
1,1433224214164,992329,view,248676,NaN,2015-06-02 05:50:14.164,2015-06-02,NaN
2,1433221999827,111016,view,318965,NaN,2015-06-02 05:13:19.827,2015-06-02,NaN
3,1433221955914,483717,view,253185,NaN,2015-06-02 05:12:35.914,2015-06-02,NaN
4,1433221337106,951259,view,367447,NaN,2015-06-02 05:02:17.106,2015-06-02,NaN
...,...,...,...,...,...,...,...,...
4439576,1438398785939,591435,view,261427,NaN,2015-08-01 03:13:05.939,2015-08-01,NaN
4439577,1438399813142,762376,view,115946,NaN,2015-08-01 03:30:13.142,2015-08-01,1616.0
4439578,1438397820527,1251746,view,78144,NaN,2015-08-01 02:57:00.527,2015-08-01,969.0
4439579,1438398530703,1184451,view,283392,NaN,2015-08-01 03:08:50.703,2015-08-01,NaN


In [33]:
grouped_data = (
    events_with_categories[events_with_categories['event'] == 'view']
    .groupby(['timestamp', 'event_datetime', 'categoryid'])
    .size()
    .reset_index(name='view_count') 
)
grouped_data

,timestamp,event_datetime,categoryid,view_count
0,1430622028399,2015-05-03 03:00:28.399,421.0,5
1,1430622028399,2015-05-03 03:00:28.399,1674.0,13
2,1430622033686,2015-05-03 03:00:33.686,1339.0,1
3,1430622036210,2015-05-03 03:00:36.210,368.0,1
4,1430622040988,2015-05-03 03:00:40.988,683.0,1
...,...,...,...,...
1459107,1442545134195,2015-09-18 02:58:54.195,1196.0,1
1459108,1442545152365,2015-09-18 02:59:12.365,1293.0,1
1459109,1442545153842,2015-09-18 02:59:13.842,956.0,1
1459110,1442545174109,2015-09-18 02:59:34.109,491.0,1


### Time series forecasting with gradient boosting: LightGBM, XGBoost 

Tree-based machine learning methods designed to improve gradient boosting decision trees (GBDT) --- for regression and classification tasks.

Key benefits of using gradient boosting models for forecasting include:

- The ease with which exogenous variables can be included in the model, in addition to autoregressive variables.
- The ability to capture non-linear relationships between variables.
- High scalability, allowing models to handle large volumes of data.
- Some implementations allow the inclusion of categorical variables without the need for additional encoding, such as one-hot encoding.

Challenges of using boosting techniques for forecasting:

- Transforming the data so that it can be used as a regression problem.
- Depending on how many future predictions are needed (prediction horizon), an iterative process may be required where each new prediction is based on previous ones.
- Model validation requires specific strategies such as backtesting, walk-forward validation or time series cross-validation. Traditional cross-validation cannot be used.

### LightGBM

LightGBM is a gradient boosting framework that uses tree based learning algorithms. It is designed to be distributed and efficient with the following advantages:

Faster training speed and higher efficiency.
- Lower memory usage.
- Better accuracy.
- Support of parallel, distributed, and GPU learning.
- Capable of handling large-scale data.

LightGBM builds decision trees iteratively. Key feature: Uses leaf-wise growth, where the tree expands its most significant leaf nodes first. This can result in deeper, more focused trees and faster convergence.

Optimizations:
- Handles large datasets efficiently.
- Uses a histogram-based algorithm for faster computation.
- Includes mechanisms for dealing with categorical features natively.

### Time series forecasting requirements: supervised learning

While boosting techniques can be used for time series forecasting, it requires that the time series dataset be transformed into a supervised learning problem first. It also requires the use of a specialized technique for evaluating the model called walk-forward validation, as evaluating the model using k-fold cross validation would result in optimistically biased results.

### Walk-forward validation

In walk-forward validation, the dataset is first split into train and test sets by selecting a cut point, e.g. all data except the last 12 days is used for training and the last 12 days is used for testing.

- Splitting the data into multiple overlapping train-validation folds, where each fold includes:
    - A growing training set (e.g., using data from t=0 to t=n).
    - A validation set that follows the training set (e.g., data from t=n+1 to t=n+m).
Training and evaluating the model on each fold.

#### One-fold validation

In [35]:
# Train-Test Split
train, test = train_test_split(grouped_data, test_size=0.2, random_state=42, shuffle=False)

first_date = test['event_datetime'].min()
last_date = test['event_datetime'].max()

print(f"Testing period: {first_date} to {last_date}")

# Assumption: the validation set should be the same length as the test set 
# and immediately precede it
validation_duration = last_date - first_date
val_start_date = first_date - validation_duration
val_end_date = first_date - pd.Timedelta(seconds=1)

# Filter the train set to define the validation set
val = train[(train['event_datetime'] >= val_start_date) & (train['event_datetime'] <= val_end_date)]

print(f"Validation period: {val_start_date} to {val_end_date}")
print(f"Number of records in validation set: {len(val)}")

# Remove validation set from training set to prevent data leakage
train = train[~train.index.isin(val.index)]

Testing period: 2015-08-17 22:28:00.761000 to 2015-09-18 02:59:41.778000
Validation period: 2015-07-17 17:56:19.744000 to 2015-08-17 22:27:59.761000
Number of records in validation set: 330165


In [36]:
# Define features and labels
features = ['timestamp', 'categoryid']  # Use the numerical timestamp
Y_train = train['view_count']
X_train = train[features]
X_train['categoryid'] = X_train['categoryid'].astype("category")

Y_val = val['view_count']
X_val = val[features]
X_val['categoryid'] = X_val['categoryid'].astype("category")

Y_test = test['view_count']
X_test = test[features]
X_test['categoryid'] = X_test['categoryid'].astype("category")

# LightGBM Dataset
lgbtrain = lgb.Dataset(data=X_train, label=Y_train, feature_name=features)
lgbval = lgb.Dataset(data=X_val, label=Y_val, reference=lgbtrain, feature_name=features)

# Define RMSLE Metric
def rmsle(y_pred, y_true):
    return np.sqrt(np.mean(np.square(np.log1p(y_pred) - np.log1p(y_true))))

def lgbm_rmsle(preds, train_data):
    labels = train_data.get_label()
    rmsle_val = rmsle(preds, labels)
    return 'RMSLE', rmsle_val, False

# LightGBM Parameters
lgb_params = {
    'num_leaves': 15,
    'learning_rate': 0.05,
    'feature_fraction': 0.8,
    'max_depth': 6,
    'verbose': 1,
    'num_boost_round': 5000,
    'early_stopping_rounds': 100,
    'nthread': -1,
}

# Train LightGBM Model
model = lgb.train(
    lgb_params,
    lgbtrain,
    valid_sets=[lgbtrain, lgbval],
    feval=lgbm_rmsle
)

# Predictions on Validation Set
y_pred_val = model.predict(X_val, num_iteration=model.best_iteration)
val_rmsle = rmsle(y_pred_val, Y_val)
print(f"Validation RMSLE: {val_rmsle}")

# Predictions on Test Set
test_preds = model.predict(X_test, num_iteration=model.best_iteration)
test_rmsle = rmsle(test_preds, Y_test)
print(f"Test RMSLE: {test_rmsle}")

# Output predictions
test['predicted_view_count'] = test_preds
print(test[['event_datetime', 'categoryid', 'view_count', 'predicted_view_count']].head())

/tmp/ipykernel_407029/1327881874.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['categoryid'] = X_train['categoryid'].astype("category")
/tmp/ipykernel_407029/1327881874.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val['categoryid'] = X_val['categoryid'].astype("category")
/tmp/ipykernel_407029/1327881874.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the do

[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.025857 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 960
[LightGBM] [Info] Number of data points in the train set: 837124, number of used features: 2
[LightGBM] [Info] Start training from score 2.082992
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[55]	training's l2: 2.7401	training's RMSLE: 0.294389	valid_1's l2: 2.73785	valid_1's RMSLE: 0.295956
Validation RMSLE: 0.2959559444152678
Test RMSLE: 0.3028402919334681
                 event_datetime  categoryid  view_count  predicted_view_count
1167289 2015-08-17 22:28:00.761       195.0           1              1.401071
11672

### XGBoost (Extreme Gradient Boosting)

XGBoost builds decision trees sequentially, improving the predictions of the previous trees. Key feature: Uses level-wise growth, which grows the tree evenly level by level.

Optimizations:
- Regularization techniques (L1, L2) to prevent overfitting.
- Support for distributed computing.
- Highly customizable hyperparameters.

In [51]:
# Convert data to DMatrix for XGBoost
dtrain = xgb.DMatrix(X_train, label=Y_train, enable_categorical=True)
dval = xgb.DMatrix(X_val, label=Y_val, enable_categorical=True)
dtest = xgb.DMatrix(X_test, enable_categorical=True)

# Define RMSLE metric
def rmsle(y_pred, y_true):
    return np.sqrt(np.mean(np.square(np.log1p(y_pred) - np.log1p(y_true))))

# XGBoost Parameters
xgb_params = {
    'objective': 'reg:squarederror',  # For regression
    'eval_metric': 'rmse',
    'max_depth': 6,
    'eta': 0.05,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'lambda': 1.0,
    'alpha': 0.0,
    'verbosity': 1,
    'seed': 42
}

# Train the model with early stopping
evals = [(dtrain, 'train'), (dval, 'val')]
model = xgb.train(
    xgb_params,
    dtrain,
    num_boost_round=5000,
    early_stopping_rounds=100,
    evals=evals,
    verbose_eval=100
)

# Predictions and Evaluation on Validation Set
y_pred_val = model.predict(dval)
val_rmsle = rmsle(y_pred_val, Y_val)
print(f"Validation RMSLE: {val_rmsle}")

# Predictions and Evaluation on Test Set
y_pred_test = model.predict(dtest)
test_rmsle = rmsle(y_pred_test, Y_test)
print(f"Test RMSLE: {test_rmsle}")

# Output predictions
test['predicted_view_count'] = y_pred_test
print(test[['event_datetime', 'categoryid', 'view_count', 'predicted_view_count']].head())

[0]	train-rmse:2.78063	val-rmse:2.66536
[100]	train-rmse:1.62335	val-rmse:1.63084
[200]	train-rmse:1.61252	val-rmse:1.63146
[217]	train-rmse:1.61249	val-rmse:1.63171
Validation RMSLE: 0.29024283661425554
Test RMSLE: 0.2955887061420854
                 event_datetime categoryid  view_count  predicted_view_count
1167289 2015-08-17 22:28:00.761      195.0           1              1.293828
1167290 2015-08-17 22:28:00.923      282.0           4              4.052262
1167291 2015-08-17 22:28:00.923      929.0          14             10.863013
1167292 2015-08-17 22:28:05.159       84.0           1              0.972632
1167293 2015-08-17 22:28:13.380      769.0           1              0.970079


/tmp/ipykernel_407029/2631139457.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['predicted_view_count'] = y_pred_test
